# Alinhamento de sequências

O alinhamento de sequências é uma ferramenta útil em vários contextos, como é o contexto do processamento de linguagem natural e previsão de escrita (i.e. autocorrect). No contexto da bioinformática, o alinhamento de sequências assume um papel preponderante para a identificação de sequências similares ou domínios conservados (seja em contexto de genómica seja em contexto de proteómica).

Permite-nos fazer assunções e correlações entre sequência, estrutura e função.

Eixstem um conjunto de abordagens a esta temática, praticamente todas envolvendo à comparação das sequências numa matrix e ir comparando elemento a elemento atribuindo valores a ocorrências idênticas (match), não-ocorrências (mismatch) e em certos cenários espaços vazios (gaps).

Em programação este problema pode ser resolvido utilizando técnicas de programação dinâmica que, apesar de eficazes, são muito pouco eficientes e custosas em termos de recursos da máquina.

Surgem então dois algoritmos comummente utilizados:

- O algoritmo de Needleman-Wunch - utilizado em contexto de alinhamento global

- Smith-Waterman - como uma implementação do algoritmo Needleman-Wucnh mas com o acréscimo de mais uma camada, que substitui todos os valores negativos por 0, sendo um excelente algoritmo para alinhamento local / refinado.

Apresentamos abaixo as explicações descritivas e prescritivas para estes algoritmos bem como a sua implementação em Python.

## Algoritmos de alinhamento comummente utilizados: Needleman-Wunsch e Smith Waterman

### Needleman-Wunch

O Algoritmo de Needleman-Wunch alinha duas sequências comparando comparando-as, elemento a elemento. Para este processo é feita uma matriz, chamada matriz de substituição, que é preenchida determinando a igualdade ou desigualdade dos elementos e atribuindo uma pontuação em função da sua igualdade. Em caso de desigualdade é contabilizado o custo de substituir um elemento ou adicionar um espaço vazio.

Abaixo apresenta-se de forma mais visual este processo, comparando a sequência AT com AT (igualdade) e AT com AC (desigualdade).

1. A matriz de substituição é criada colocando em comparação as duas sequências e adicionando um espaço em cada uma, para consideração. As células correspondentes a estes espaços são preenchadas com a soma consecutiva da adição de um espaço ('caminhando' para a direita ou para baixo)
<table>
<tr>
    <td></td>
    <td>-</td>
    <td>A</td>
    <td>T</td>
</tr>
<tr>
    <td>-</td>
    <td>0</td>
    <td>-4</td>
    <td>-8</td>
</tr>
<tr>
    <td>A</td>
    <td>-4</td>
    <td></td>
    <td></td>
</tr>
<tr>
    <td>T</td>
    <td>-8</td>
    <td></td>
    <td></td>
</tr>
</table>


2. É feito então a comparação dos elementos. 
    * No sentido horizontal ou vertical é adicionado o valor atribuído à adição de um espaço (-4 -4 = -8 e -4 -4 = -8 ).
    * No sentido diagonal é feita a comparação dos elementos e adicionado o valor atribuído à substituição (em caso de diferença) ou à igualdade (no caso de serem elementos iguais). Neste caso, assumindo um valor de igualdade de +1 e considerando que ambos os elementos são iguais (A == A) adicionamos +1 e obtemos desta forma (0 +1 = 1).
    * Por fim é mantido o valor maior, que será o que populará a célula da matriz correspondente à comparação destes dois elementos.
    * E assim por diante até chegarmos ao final da matriz.

<table>
<tr>
    <td></td>
    <td>-</td>
    <td>A</td>
    <td>T</td>
</tr>
<tr>
    <td>-</td>
    <td>0</td>
    <td>-4</td>
    <td>-8</td>
</tr>
<tr>
    <td>A</td>
    <td>-4</td>
    <td>1 (D)</td>
    <td>-3</td>
</tr>
<tr>
    <td>T</td>
    <td>-8</td>
    <td>-3</td>
    <td>2 (D)</td>
</tr>
</table>

Alinhamento ótimo:

AT
AT

3. Por fim, consideramos o score final do alinhamento (correspondente ao canto inferior da nossa matriz de substituição) e passamos à construção da nossa matriz de traceback para encontrar o alinhamento ótimo. A lógica para o traceback é a seguinte:
    * Caso o elemento seja igual, percorremos a matriz diagonalmente;
    * Caso o elemento seja diferente, percorremos vertical ou horizontalmente no sentido do maior valor. Quando é feito o percurso horizontal ou verticalmente é acrescentado um espaço à segunda ou primeira sequência alinhada respetivamente.
    * O exemplo infra demonstra este caso, ao alinhar sequências diferentes ('ATCG com ATCC')
    

 <table>
<tr>
    <td></td>
    <td>-</td>
    <td>A</td>
    <td>T</td>
    <td>C</td>
    <td>G</td>
</tr>
<tr>
    <td>-</td>
    <td>0</td>
    <td>-4</td>
    <td>-8</td>
    <td>-12</td>
    <td>-16</td>
</tr>
<tr>
    <td>A</td>
    <td>-4</td>
    <td>1 (D)</td>
    <td>-3</td>
    <td>-8</td>
    <td>-8</td>
</tr>
<tr>
    <td>T</td>
    <td>-8</td>
    <td>-3</td>
    <td>2 (D)</td>
    <td>-2 (C)</td>
    <td>-6</td>
</tr>
<tr>
    <td>C</td>
    <td>-12</td>
    <td>-7</td>
    <td>-2</td>
    <td>3 (D)</td>
    <td>-1 (D)</td>
</tr>
<tr>
    <td>C</td>
    <td>-16</td>
    <td>-11</td>
    <td>-6</td>
    <td>-1</td>
    <td>2 (C/E)</td>
</tr>
</table>   


Alinhamentos Ótimos

1: 

ATC-
ATC-

2: 

ATC-
AT-C



In [3]:
def needleman_wunsch(string_1 : str, string_2 : str, equal = 2 ,subst = -1, score_space : int = -4) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''

  from scripts.auxiliares import validar_dna, aprimorar_seq
  from scripts.alinhamentos import score_subst, print_matrix, reconstroi
  
  


  
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str
  assert validar_dna(string_1) and validar_dna(string_2)
  assert string_1 and string_2 != '' or ' '

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + aprimorar_seq(string_1)
  string_2 = '-' + aprimorar_seq(string_2)

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]

  # Inicializar a primeira linha
  # Atribuí-se o valor de penalidade porque a primeira linha corresponde apenas a alinhamento espaços vazios
  # (ver output para visualizar)
  score[0] = [posicao_coluna * score_space for posicao_coluna, elemento in enumerate(string_1)]

  # Como estamos na primeira linha, estamos a alinhar apenas com espaços vazios
  # pelo que a matriz traceback indicará que devemos sempre caminhar à esquerda
  # (ver output para visualizar)

  matriz_traceback[0] = [0 if posicao_coluna == 0 else 'E' for posicao_coluna, elemento in enumerate(string_1)]


  # Inicializar a primeira coluna
  
  for posicao_linha, elemento in enumerate(string_2):
  
    # A lógica de preenchimento é semelhante à das linhas, mudando apenas 
    # a direção, que será vertical

    # Iteramos sobre o indice correspondente à posição do elemento na linha, indice 0 que corresponde
    # a primeira coluna 
    score[posicao_linha][0] = posicao_linha * score_space
  
    # Conforme mencionado acima, mesma lógica mas na vertical
    matriz_traceback[posicao_linha][0] = 0 if posicao_linha == 0 else 'C'


  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):

    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):

      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.

        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2, equal, subst, score_space)
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor

        # Convertemos em direção e populamos a matriz traceback
        matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)]

        

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''

  return score[-1][-1], reconstroi(string_1, string_2, score, matriz_traceback, 'nw')

In [18]:
needleman_wunsch('ATCG','ATCC', 1,-1,-4)

    -   A   T   C   G
-   0  -4  -8 -12 -16
A  -4   1  -3  -7 -11
T  -8  -3   2  -2  -6
C -12  -7  -2   3  -1
C -16 -11  -6  -1   2

    -   A   T   C   G
-   0   E   E   E   E
A   C   D   E   E   E
T   C   C   D   E   E
C   C   C   C   D   E
C   C   C   C   D   D


ATCG
ATCC


(2, ('ATCG', 'ATCC'))

In [8]:
# Testes de unidade

import unittest

# Secção de testes de unidade

class TestNeedlemanWunch(unittest.TestCase):
    
    def test_sequencia_vazia(self):   
        with self.assertRaises(AssertionError):
            needleman_wunsch('',' ')

    def test_framgentos(self):
        self.assertEqual( needleman_wunsch('ATGCGTCGA','aagta') , (-9, ('ATGCGTCGA', 'A--AGT--A')))

    def test_apenas_um_match(self):
        self.assertEqual( needleman_wunsch('GGCATGCG','A') , (-26, ('GGCATGCG', '---A----')))

    def test_nao_dna(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch('HGJK','ATC')

    def test_simbolos(self):
        with self.assertRaises(AssertionError):
            needleman_wunsch('A--GCTG--ACG','AGTG$%$CACG')

    def test_subfuncoes(self):
        self.assertEqual(needleman_wunsch('aacgt','AACGT'), (10, ('AACGT', 'AACGT')))

suite = unittest.TestLoader().loadTestsFromTestCase(TestNeedlemanWunch)
unittest.TextTestRunner( verbosity=3 ).run( suite )


test_apenas_um_match (__main__.TestNeedlemanWunch.test_apenas_um_match) ... ok
test_framgentos (__main__.TestNeedlemanWunch.test_framgentos) ... ok
test_nao_dna (__main__.TestNeedlemanWunch.test_nao_dna) ... ok
test_sequencia_vazia (__main__.TestNeedlemanWunch.test_sequencia_vazia) ... ok
test_simbolos (__main__.TestNeedlemanWunch.test_simbolos) ... ok
test_subfuncoes (__main__.TestNeedlemanWunch.test_subfuncoes) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK


    -   G   G   C   A   T   G   C   G
-   0  -4  -8 -12 -16 -20 -24 -28 -32
A  -4  -1  -5  -9 -10 -14 -18 -22 -26

    -   G   G   C   A   T   G   C   G
-   0   E   E   E   E   E   E   E   E
A   C   D   D   D   D   E   E   E   E


GGCATGCG
---A----
    -   A   T   G   C   G   T   C   G   A
-   0  -4  -8 -12 -16 -20 -24 -28 -32 -36
A  -4   2  -2  -6 -10 -14 -18 -22 -26 -30
A  -8  -2   1  -3  -7 -11 -15 -19 -23 -24
G -12  -6  -3   3  -1  -5  -9 -13 -17 -21
T -16 -10  -4  -1   2  -2  -3  -7 -11 -15
A -20 -14  -8  -5  -2   1  -3  -4  -8  -9

    -   A   T   G   C   G   T   C   G   A
-   0   E   E   E   E   E   E   E   E   E
A   C   D   E   E   E   E   E   E   E   D
A   C   D   D   D   D   D   D   D   D   D
G   C   C   D   D   E   D   E   E   D   E
T   C   C   D   C   D   D   D   E   E   E
A   C   D   C   D   D   D   D   D   D   D


ATGCGTCGA
A--AGT--A
    -   A   A   C   G   T
-   0  -4  -8 -12 -16 -20
A  -4   2  -2  -6 -10 -14
A  -8  -2   4   0  -4  -8
C -12  -6   0   6   2  -2
G -16 -10 

<unittest.runner.TextTestResult run=6 errors=0 failures=0>

### Smith Waterman

O algoritmo de Smith-Waterman segue a mesma lógica que o algoritmo de Needleman-Wunsch com a diferença de que, caso o score a atribuir seja negativo é sempre atribuído o valor 0. 

Esta abordagem torna este algoritmo menos apto para alinhamento global, mas uma excelente opção para a identificação de diferentes alinhamentos ótimos localmente (e.g. domínios conservados).

In [14]:
def smith_waterman(string_1 : str, string_2 : str, equal = 1, subst = -1, score_space : int = -4) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''

  from scripts.auxiliares import validar_dna, aprimorar_seq
  from scripts.alinhamentos import score_subst, print_matrix, reconstroi
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str
  assert validar_dna(string_1) and validar_dna(string_2)
  assert string_1 and string_2 != '' or ' '

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + aprimorar_seq(string_1)
  string_2 = '-' + aprimorar_seq(string_2)

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]
  max_score = max([val for linha in score for val in linha])

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]



  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):
    #print('#####linha',posicao_linha,'#####') 
    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):
      #print('####coluna',posicao_coluna,'####')
      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.
        
        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2, equal, subst, score_space)
        #print('diag',diag)
        #print('e1,e2',(elemento_string_1,elemento_string_2))
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space
        #print('esq',esq,'score-space',score_space)

        

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space
        #print('cima',cima)
        

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima, 0]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        #print('valor=',valor)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor


        # Convertemos em direção e populamos a matriz traceback
        if valor != 0: matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)] 
      
        # else: matriz_traceback[posicao_linha][posicao_coluna] = 0

     

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''
  # Smith Waterman parte do valor mais alto para a reconstrução, por isso o valor de partida tem de ser o maior da matriz
  # Para isso é preciso "espalmar" a matriz para encontrar o valor mais alto

  max_score = max([val for linha in score for val in linha])
  # print('pontuacao-maxima=',max_score)
  return max_score, reconstroi(string_1, string_2, score, matriz_traceback, 'sw')

In [15]:
smith_waterman('AT','ATCGTAT')

    -   A   T
-   0   0   0
A   0   1   0
T   0   0   2
C   0   0   0
G   0   0   0
T   0   0   1
A   0   1   0
T   0   0   2

    -   A   T
-   0   0   0
A   0   D   0
T   0   0   D
C   0   0   0
G   0   0   0
T   0   0   D
A   0   D   0
T   0   0   D


AT
AT


(2, ('AT', 'AT'))

In [17]:
# Secção de testes de unidade

class TestSmithWaterman(unittest.TestCase):
    
    def test_sequencia_vazia(self):   
        with self.assertRaises(AssertionError):
            smith_waterman('',' ')

    def test_sem_matches_possiveis(self):
        self.assertEqual( smith_waterman('GTC','AAA') , (0, ('', '')))

    def test_apenas_um_match(self):
        self.assertEqual( smith_waterman('GGCATGCG','AAA') , (1, ('A', 'A')))

    def test_nao_dna(self):
        with self.assertRaises(AssertionError):
            smith_waterman('HGJK','ATC')

    def test_simbolos(self):
        with self.assertRaises(AssertionError):
            smith_waterman('A--GCTG--ACG','AGTG$%$CACG')

    def test_subfuncoes(self):
        self.assertEqual(smith_waterman('aacgt','AACGT'), (5, ('AACGT', 'AACGT')))

suite = unittest.TestLoader().loadTestsFromTestCase(TestSmithWaterman)
unittest.TextTestRunner( verbosity=3 ).run( suite )


test_apenas_um_match (__main__.TestSmithWaterman.test_apenas_um_match) ... ok
test_nao_dna (__main__.TestSmithWaterman.test_nao_dna) ... ok
test_sem_matches_possiveis (__main__.TestSmithWaterman.test_sem_matches_possiveis) ... ok
test_sequencia_vazia (__main__.TestSmithWaterman.test_sequencia_vazia) ... ok
test_simbolos (__main__.TestSmithWaterman.test_simbolos) ... ok
test_subfuncoes (__main__.TestSmithWaterman.test_subfuncoes) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK


    -   G   G   C   A   T   G   C   G
-   0   0   0   0   0   0   0   0   0
A   0   0   0   0   1   0   0   0   0
A   0   0   0   0   1   0   0   0   0
A   0   0   0   0   1   0   0   0   0

    -   G   G   C   A   T   G   C   G
-   0   0   0   0   0   0   0   0   0
A   0   0   0   0   D   0   0   0   0
A   0   0   0   0   D   0   0   0   0
A   0   0   0   0   D   0   0   0   0


A
A
    -   G   T   C
-   0   0   0   0
A   0   0   0   0
A   0   0   0   0
A   0   0   0   0

    -   G   T   C
-   0   0   0   0
A   0   0   0   0
A   0   0   0   0
A   0   0   0   0




    -   A   A   C   G   T
-   0   0   0   0   0   0
A   0   1   1   0   0   0
A   0   1   2   0   0   0
C   0   0   0   3   0   0
G   0   0   0   0   4   0
T   0   0   0   0   0   5

    -   A   A   C   G   T
-   0   0   0   0   0   0
A   0   D   D   0   0   0
A   0   D   D   0   0   0
C   0   0   0   D   0   0
G   0   0   0   0   D   0
T   0   0   0   0   0   D


AACGT
AACGT


<unittest.runner.TextTestResult run=6 errors=0 failures=0>